### ACCESSING DATA

#### Data Quality and Tidiness Issues

In [1]:
import pandas as pd


df_allseasons = pd.read_csv('cleaned_merged_seasons.csv')

df_allseasons.head()

/home/laniolao/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Unnamed: 0,season_x,name,position,team_x,assists,bonus,bps,clean_sheets,creativity,...,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,0,2016-17,Aaron Cresswell,DEF,NaN,0,0,0,0,0.0,...,2.0,0.0,0,0,0,0,55,False,0,1
1,1,2016-17,Aaron Lennon,MID,NaN,0,0,6,0,0.3,...,1.0,0.0,1,0,0,0,60,True,0,1
2,2,2016-17,Aaron Ramsey,MID,NaN,0,0,5,0,4.9,...,3.0,23.0,2,0,0,0,80,True,0,1
3,3,2016-17,Abdoulaye Doucouré,MID,NaN,0,0,0,0,0.0,...,1.0,0.0,0,0,0,0,50,False,0,1
4,4,2016-17,Adam Forshaw,MID,NaN,0,0,3,0,1.3,...,1.0,0.0,1,0,0,0,45,True,1,1


### CLEANING DATA

In [60]:
# Make a copy of the original piece of data.
df_allseasons_clean = df_allseasons.copy()

In [61]:
df_allseasons_clean.drop(['Unnamed: 0', 'team_x'], axis=1, inplace=True)

In [62]:
df_allseasons_clean['name_season'] = df_allseasons_clean['name'] + '_' + df_allseasons_clean['season_x']

In [63]:
df_allseasons_clean.name_season

0           Aaron Cresswell_2016-17
1              Aaron Lennon_2016-17
2              Aaron Ramsey_2016-17
3        Abdoulaye Doucouré_2016-17
4              Adam Forshaw_2016-17
                    ...            
98397         Wilfred Ndidi_2021-22
98398          Matt Ritchie_2021-22
98399        Nathan Redmond_2021-22
98400           Mathew Ryan_2021-22
98401       Ryan Fredericks_2021-22
Name: name_season, Length: 98402, dtype: object

In [64]:
import requests

# get yearly historic data from endpoint for available seasons and identify the keys in each disctionary using 2016 as an example
Y2016= requests.get('https://www.fantasynutmeg.com/api/history/season/2016-17').json()
Y2017= requests.get('https://www.fantasynutmeg.com/api/history/season/2017-18').json()
Y2018= requests.get('https://www.fantasynutmeg.com/api/history/season/2018-19').json()
Y2019= requests.get('https://www.fantasynutmeg.com/api/history/season/2019-20').json()
Y2020= requests.get('https://www.fantasynutmeg.com/api/history/season/2020-21').json()
Y2021= requests.get('https://www.fantasynutmeg.com/api/history/season/2021-22').json()
Y2022= requests.get('https://www.fantasynutmeg.com/api/history/season/2022-23').json()
Y2016.keys()

dict_keys(['dd_agg_fixture', 'dd_agg_player', 'dd_hauls', 'history'])

Hist from 16-21.
Create a column with the First Name, Last Name and the Year.
Combiine the name and the year of the cloned repo to match the hist data.
Merge the two based on the First Name, Last Name and the Year. 

In [65]:
hist16_df = pd.DataFrame(Y2016['history'])
hist17_df = pd.DataFrame(Y2017['history'])
hist18_df = pd.DataFrame(Y2018['history'])
hist19_df = pd.DataFrame(Y2019['history'])
hist20_df = pd.DataFrame(Y2020['history'])
hist21_df = pd.DataFrame(Y2021['history'])

In [66]:
hist16_df['year'] = hist16_df.apply(lambda x: "2016-17", axis=1)
hist17_df['year'] = hist17_df.apply(lambda x: "2017-18", axis=1)
hist18_df['year'] = hist18_df.apply(lambda x: "2018-19", axis=1)
hist19_df['year'] = hist19_df.apply(lambda x: "2019-20", axis=1)
hist20_df['year'] = hist20_df.apply(lambda x: "2020-21", axis=1)
hist21_df['year'] = hist21_df.apply(lambda x: "2021-22", axis=1)

In [67]:
hist_df = [hist16_df, hist17_df, hist18_df, hist19_df, hist20_df, hist21_df]

hist = pd.concat(hist_df, axis = 0, ignore_index=True)

In [68]:
hist.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,...,influence_rank,influence_rank_type,threat_rank,threat_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text
0,0,0,18,100,100,0,48844,0,0,-3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,2,660,100,100,12,11334,0,0,-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,19,723,0,75,10,51507,0,0,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,5,100,100,0,17127,0,0,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,2,296,75,100,5,158074,0,0,-2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
hist['form'] = hist['total_points']/38    # Number of gameweeks

In [70]:
hist.columns

Index(['assists', 'bonus', 'bps', 'chance_of_playing_next_round',
       'chance_of_playing_this_round', 'clean_sheets', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'creativity', 'dreamteam_count', 'ea_index',
       'element_type', 'ep_next', 'ep_this', 'event_points', 'first_name',
       'form', 'goals_conceded', 'goals_scored', 'ict_index', 'id',
       'in_dreamteam', 'influence', 'loaned_in', 'loaned_out', 'loans_in',
       'loans_out', 'minutes', 'news', 'now_cost', 'own_goals',
       'penalties_missed', 'penalties_saved', 'photo', 'points_per_game',
       'position', 'red_cards', 'saves', 'second_name', 'selected_by_percent',
       'special', 'squad_number', 'status', 'team', 'team_code', 'team_name',
       'threat', 'total_points', 'transfers_in', 'transfers_in_event',
       'transfers_out', 'transfers_out_event', 'value_form', 'value_season',
       'web_name', 'yellow_cards', 'year', 'news_added', 

In [71]:
hist['name_season'] = hist['first_name'] + ' ' + hist['second_name'] + '_' + hist['year']

In [72]:
hist.name_season

0                   David Ospina_2016-17
1                      Petr Cech_2016-17
2              Laurent Koscielny_2016-17
3                Per Mertesacker_2016-17
4       Gabriel Armando de Abreu_2016-17
                      ...               
4065                Adrian Blake_2021-22
4066                Jack Grieves_2021-22
4067              Joseph McGlynn_2021-22
4068             Anthony Mancini_2021-22
4069              Charlie Savage_2021-22
Name: name_season, Length: 4070, dtype: object

In [73]:
subset  = ['Mohamed Salah_2018-19']
check = hist[hist.name_season.isin(subset)]
check.form

1582    6.815789
Name: form, dtype: float64

In [74]:
teams=dict(zip(hist.name_season, hist.team_name))

df_allseasons_clean['club_name'] = df_allseasons_clean['name_season'].map(teams)

In [75]:
teams=dict(zip(hist.name_season, hist.form))

df_allseasons_clean['form'] = df_allseasons_clean['name_season'].map(teams)

In [76]:
subset  = ['Marcus Rashford_2020-21']
check = df_allseasons_clean[df_allseasons_clean.name_season.isin(subset)]
check.form

50237    4.578947
50787    4.578947
51344    4.578947
51917    4.578947
52611    4.578947
53207    4.578947
53807    4.578947
54408    4.578947
55010    4.578947
55566    4.578947
56157    4.578947
56764    4.578947
57374    4.578947
57988    4.578947
58498    4.578947
59044    4.578947
59480    4.578947
60327    4.578947
60328    4.578947
61026    4.578947
61674    4.578947
62336    4.578947
63001    4.578947
63780    4.578947
64523    4.578947
65608    4.578947
65609    4.578947
65890    4.578947
66643    4.578947
67606    4.578947
68293    4.578947
68985    4.578947
69666    4.578947
70880    4.578947
70881    4.578947
70882    4.578947
72475    4.578947
73181    4.578947
Name: form, dtype: float64

In [77]:
df_allseasons_clean.columns

Index(['season_x', 'name', 'position', 'assists', 'bonus', 'bps',
       'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time', 'minutes',
       'opponent_team', 'opp_team_name', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW', 'name_season', 'club_name', 'form'],
      dtype='object')

In [82]:
df_allseasons_clean[df_allseasons_clean.minutes == 0]

,season_x,name,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW,name_season,club_name,form
0,2016-17,Aaron Cresswell,DEF,0,0,0,0,0.0,454,10,...,0,0,0,55,False,0,1,Aaron Cresswell_2016-17,WHU,1.578947
3,2016-17,Abdoulaye Doucouré,MID,0,0,0,0,0.0,482,7,...,0,0,0,50,False,0,1,Abdoulaye Doucouré_2016-17,WAT,1.000000
8,2016-17,Alex McCarthy,GK,0,0,0,0,0.0,101,7,...,0,0,0,45,True,0,1,Alex McCarthy_2016-17,SOU,0.000000
10,2016-17,Andreas Pereira,MID,0,0,0,0,0.0,263,9,...,0,0,0,45,False,0,1,Andreas Pereira_2016-17,MUN,0.000000
15,2016-17,Angelo Ogbonna,DEF,0,0,0,0,0.0,456,10,...,0,0,0,50,False,0,1,Angelo Ogbonna_2016-17,WHU,1.184211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98393,2021-22,Jack Grieves,FWD,0,0,0,0,0.0,734,375,...,4094,5073,979,45,False,0,38,Jack Grieves_2021-22,WAT,0.000000
98396,2021-22,Josh Martin,MID,0,0,0,0,0.0,330,380,...,0,0,0,50,True,0,38,Josh Martin_2021-22,NOR,0.000000
98397,2021-22,Wilfred Ndidi,MID,0,0,0,0,0.0,216,377,...,-202,22,224,48,True,0,38,Wilfred Ndidi_2021-22,LEI,1.078947
98400,2021-22,Mathew Ryan,GK,0,0,0,0,0.0,65,373,...,-2,0,2,45,True,0,38,Mathew Ryan_2021-22,BHA,0.000000


In [83]:
zero_minutes = df_allseasons_clean[df_allseasons_clean.minutes == 0].index

In [84]:
df_allseasons_clean.drop(zero_minutes, axis = 0, inplace=True)

In [85]:
df_allseasons_clean.drop(['round'], axis=1, inplace=True)

In [86]:
df_allseasons_clean.kickoff_time

1        2016-08-13T14:00:00Z
2        2016-08-14T15:00:00Z
4        2016-08-13T14:00:00Z
5        2016-08-14T15:00:00Z
6        2016-08-15T19:00:00Z
                 ...         
98392    2022-05-22T15:00:00Z
98394    2022-05-22T15:00:00Z
98395    2022-05-22T15:00:00Z
98398    2022-05-22T15:00:00Z
98399    2022-05-22T15:00:00Z
Name: kickoff_time, Length: 49231, dtype: object

In [87]:
df_allseasons_clean['game_date'] = df_allseasons_clean['kickoff_time'].str.replace('T', ' ')
df_allseasons_clean['game_date'] = df_allseasons_clean['game_date'].str.replace(':00Z', '')

In [88]:
df_allseasons_clean['game_date'] = pd.to_datetime(df_allseasons_clean['game_date'])

In [89]:
df_allseasons_clean.game_date

1       2016-08-13 14:00:00
2       2016-08-14 15:00:00
4       2016-08-13 14:00:00
5       2016-08-14 15:00:00
6       2016-08-15 19:00:00
                ...        
98392   2022-05-22 15:00:00
98394   2022-05-22 15:00:00
98395   2022-05-22 15:00:00
98398   2022-05-22 15:00:00
98399   2022-05-22 15:00:00
Name: game_date, Length: 49231, dtype: datetime64[ns]

In [90]:
seasons = [1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 1]

month_to_season = dict(zip(range(1,13), seasons))
month_to_season

{1: 1, 2: 1, 3: 2, 4: 2, 5: 2, 6: 3, 7: 3, 8: 3, 9: 4, 10: 4, 11: 4, 12: 1}

In [91]:
df_allseasons_clean['game_weather'] = df_allseasons_clean.game_date.dt.month.map(month_to_season) 

In [92]:
df_allseasons_clean.game_weather.value_counts()

1    18277
4    12980
2    12248
3     5726
Name: game_weather, dtype: int64

In [93]:
df_allseasons_clean.game_date.value_counts()

2022-05-22 15:00:00    280
2021-05-23 15:00:00    278
2020-07-26 15:00:00    275
2019-05-12 14:00:00    207
2018-05-13 14:00:00    175
                      ... 
2016-12-05 20:00:00      7
2016-12-10 12:30:00      7
2016-11-27 12:00:00      6
2016-10-16 12:30:00      6
2016-08-21 12:30:00      3
Name: game_date, Length: 1427, dtype: int64

In [94]:
df_allseasons_clean.game_date.dt.hour.value_counts()

15    10796
14     9845
19     6339
20     4359
16     4165
17     3741
12     3266
11     2365
18     2189
13     2166
Name: game_date, dtype: int64

In [95]:
import numpy as np
df_allseasons_clean['start_label'] = np.where((df_allseasons_clean['game_date'].dt.hour) < 13, 0, 1)

In [96]:
df_allseasons_clean.start_label.value_counts()

1    43600
0     5631
Name: start_label, dtype: int64

In [97]:
df_allseasons_clean[['game_date', 'start_label']].head(50)

,game_date,start_label
1,2016-08-13 14:00:00,1
2,2016-08-14 15:00:00,1
4,2016-08-13 14:00:00,1
5,2016-08-14 15:00:00,1
6,2016-08-15 19:00:00,1
7,2016-08-14 15:00:00,1
9,2016-08-14 15:00:00,1
11,2016-08-13 11:30:00,0
12,2016-08-13 14:00:00,1
13,2016-08-13 14:00:00,1


In [98]:
df_allseasons_clean.drop(['game_date'], axis=1, inplace=True)

df_allseasons_clean

,season_x,name,position,assists,bonus,bps,clean_sheets,creativity,element,fixture,...,transfers_out,value,was_home,yellow_cards,GW,name_season,club_name,form,game_weather,start_label
1,2016-17,Aaron Lennon,MID,0,0,6,0,0.3,142,3,...,0,60,True,0,1,Aaron Lennon_2016-17,EVE,0.578947,3,1
2,2016-17,Aaron Ramsey,MID,0,0,5,0,4.9,16,8,...,0,80,True,0,1,Aaron Ramsey_2016-17,ARS,1.473684,3,1
4,2016-17,Adam Forshaw,MID,0,0,3,0,1.3,286,6,...,0,45,True,1,1,Adam Forshaw_2016-17,MID,2.026316,3,1
5,2016-17,Adam Lallana,MID,1,2,33,0,33.7,205,8,...,0,70,False,1,1,Adam Lallana_2016-17,LIV,3.657895,3,1
6,2016-17,Adrián San Miguel del Castillo,GK,0,0,16,0,0.0,450,10,...,0,50,False,0,1,Adrián San Miguel del Castillo_2016-17,WHU,1.684211,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98392,2021-22,John Ruddy,GK,0,0,11,0,0.0,452,378,...,120,43,False,0,38,John Ruddy_2021-22,WOL,0.078947,2,1
98394,2021-22,Mohammed Salisu,DEF,0,0,13,0,0.0,351,377,...,529,45,False,0,38,Mohammed Salisu_2021-22,SOU,1.526316,2,1
98395,2021-22,N'Golo Kanté,MID,0,0,12,0,0.0,130,375,...,2468,49,True,0,38,N'Golo Kanté_2021-22,CHE,2.078947,2,1
98398,2021-22,Matt Ritchie,DEF,0,0,3,0,0.0,292,374,...,253,49,False,0,38,Matt Ritchie_2021-22,NEW,0.526316,2,1


In [99]:
df_allseasons_clean.drop(['name_season', 'season_x', 'name', 'opponent_team', 'fixture', 'kickoff_time'], axis=1, inplace=True)

df_allseasons_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49231 entries, 1 to 98399
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   position           49231 non-null  object 
 1   assists            49231 non-null  int64  
 2   bonus              49231 non-null  int64  
 3   bps                49231 non-null  int64  
 4   clean_sheets       49231 non-null  int64  
 5   creativity         49231 non-null  float64
 6   element            49231 non-null  int64  
 7   goals_conceded     49231 non-null  int64  
 8   goals_scored       49231 non-null  int64  
 9   ict_index          49231 non-null  float64
 10  influence          49231 non-null  float64
 11  minutes            49231 non-null  int64  
 12  opp_team_name      49231 non-null  object 
 13  own_goals          49231 non-null  int64  
 14  penalties_missed   49231 non-null  int64  
 15  penalties_saved    49231 non-null  int64  
 16  red_cards          492

In [100]:
# df_allseasons["club_name"].replace({"WHU": "West Ham", "EVE": "Everton", "ARS": "Arsenal", "WAT": "Watford", "MID": "Middlesbrough",
# "LIV": "Liverpool", "SOU": "Southampton", "MUN": "Man Utd", "HUL": "Hull", "BUR": "Burnley", "CRY": "Crystal Palace", "TOT": "Spurs",
# "LEI": "Leicester", "WBA": "West Brom", "CHE": "Chelsea", "BOU": "Bournemouth", "STK": "Stoke",
# "MCI": "Man City", "SWA": "Swansea", "SUN": "Sunderland", "HUD": "Huddersfield", "BHA": "Brighton",
# "NEW": "Newcastle", "FUL": "Fulham", "WOL": "Wolves", "CAR": "Cardiff", "AVL": "Aston Villa",
# "NOR": "Norwich", "SHU": "Sheffield Utd", "LEE": "Leeds", "BRE": "Brentford"}, inplace=True)

In [101]:
df_allseasons_clean.isna().sum()

position              0
assists               0
bonus                 0
bps                   0
clean_sheets          0
creativity            0
element               0
goals_conceded        0
goals_scored          0
ict_index             0
influence             0
minutes               0
opp_team_name         0
own_goals             0
penalties_missed      0
penalties_saved       0
red_cards             0
saves                 0
selected              0
team_a_score          0
team_h_score          0
threat                0
total_points          0
transfers_balance     0
transfers_in          0
transfers_out         0
value                 0
was_home              0
yellow_cards          0
GW                    0
club_name            37
form                 37
game_weather          0
start_label           0
dtype: int64

In [102]:
df_allseasons_clean.dropna(inplace=True)

In [103]:
# position
# opp_team_name
# club_name
# was_home

In [104]:
df_allseasons_clean.position = df_allseasons_clean.position.astype(str)
df_allseasons_clean.opp_team_name = df_allseasons_clean.opp_team_name.astype(str)
df_allseasons_clean.club_name = df_allseasons_clean.club_name.astype(str)
df_allseasons_clean.was_home = df_allseasons_clean.was_home.astype(str)

In [105]:
df_allseasons_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49194 entries, 1 to 98399
Data columns (total 34 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   position           49194 non-null  object 
 1   assists            49194 non-null  int64  
 2   bonus              49194 non-null  int64  
 3   bps                49194 non-null  int64  
 4   clean_sheets       49194 non-null  int64  
 5   creativity         49194 non-null  float64
 6   element            49194 non-null  int64  
 7   goals_conceded     49194 non-null  int64  
 8   goals_scored       49194 non-null  int64  
 9   ict_index          49194 non-null  float64
 10  influence          49194 non-null  float64
 11  minutes            49194 non-null  int64  
 12  opp_team_name      49194 non-null  object 
 13  own_goals          49194 non-null  int64  
 14  penalties_missed   49194 non-null  int64  
 15  penalties_saved    49194 non-null  int64  
 16  red_cards          491

In [106]:
# Seperating the features from the target
features = df_allseasons_clean.drop('total_points', axis=1)
target = df_allseasons_clean['total_points']

In [111]:
features_dict = features.to_dict(orient='records')

In [112]:
# DictVectorizer
from sklearn.feature_extraction import DictVectorizer

# instantiate a Dictvectorizer object for X

dv = DictVectorizer(sparse=False) 

# sparse = False makes the output is not a sparse matrix

features_encoded = dv.fit_transform(features_dict)

In [116]:
# vocabulary
vocab = dv.vocabulary_
# show vocab
vocab

{'position=MID': 82,
 'assists': 1,
 'bonus': 2,
 'bps': 3,
 'clean_sheets': 4,
 'creativity': 36,
 'element': 37,
 'goals_conceded': 40,
 'goals_scored': 41,
 'ict_index': 42,
 'influence': 43,
 'minutes': 44,
 'opp_team_name=Spurs': 68,
 'own_goals': 76,
 'penalties_missed': 77,
 'penalties_saved': 78,
 'red_cards': 83,
 'saves': 84,
 'selected': 85,
 'team_a_score': 87,
 'team_h_score': 88,
 'threat': 89,
 'transfers_balance': 90,
 'transfers_in': 91,
 'transfers_out': 92,
 'value': 93,
 'was_home=True': 95,
 'yellow_cards': 96,
 'GW': 0,
 'club_name=EVE': 14,
 'form': 38,
 'game_weather': 39,
 'start_label': 86,
 'opp_team_name=Liverpool': 60,
 'club_name=ARS': 5,
 'opp_team_name=Stoke': 69,
 'club_name=MID': 22,
 'opp_team_name=Arsenal': 45,
 'was_home=False': 94,
 'club_name=LIV': 20,
 'position=GK': 81,
 'opp_team_name=Chelsea': 52,
 'club_name=WHU': 34,
 'position=DEF': 79,
 'opp_team_name=Leicester': 59,
 'club_name=HUL': 17,
 'position=FWD': 80,
 'opp_team_name=Swansea': 71,


### Feature Selection (VIF)

In [117]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Utility function to return the VIF value for each feature provided
def compute_vif(features, df):
    """
    Returns a DataFrame containing features and their corresponding variance inflation factor
    features: list of features whoes multicollinearity check is needed
    df: DataFrame of the data under review
    """
    X = df[features]   
    X['intercept'] = 1
    # Create dataframe to store vif values
    vif = pd.DataFrame()
    vif['Feature'] = X.columns
    vif['Vif Factor'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif[vif['Feature']!='intercept']
    return vif

In [120]:
def select_features(df, threshold):
    """
    Returns two objects;
    1. a DataFrame containing features and their corresponding variance inflation factor, and
    2. Pandas Index object containing the list of features that have the least Multicollinearity in accordance with
       the supplied threshold.
    train_df: The training dataset whoes Multicollinearity is to be checked
    threshold: value to compare VIF value with, above which, the feature is dropped.
    """
    data = df.copy()
    flag = True
    while flag:
        features_to_consider = data.columns
        # Calling the "compute_vif" utility function the Variance Inflation Factor dataframe
        sorted_vif_df = (compute_vif(features_to_consider, data) 
                         .sort_values('Vif Factor', ascending=False).reset_index().drop('index', axis=1))

        # Get the highest vif value to compare against a threshold
        highest_vif = sorted_vif_df.at[0, 'Vif Factor']
        
        # Compare the highest_vif with a threshold (100 was decided for this problem by the team)
        if highest_vif >= threshold: # or highest_vif=='inf':
            # Select the feature corresponding to the highest_vif (index 0 for both)
            feature = sorted_vif_df.at[0, 'Feature'] 
            # Drop the feature
            data.drop(feature, axis=1, inplace=True) 
            
        else:
            flag = False
    return sorted_vif_df#, data.columns

In [128]:
df = pd.DataFrame(i for i in features_encoded).transpose()

In [ ]:
df = pd.DataFrame(features_encoded, columns = )

In [129]:
df.drop(0, axis = 1, inplace=True)
df.columns = features_encoded[0]

,0,1,2,3,4,5,6,7,8,9,...,49184,49185,49186,49187,49188,49189,49190,49191,49192,49193
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0,38.0
1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6.0,5.0,3.0,33.0,16.0,12.0,23.0,14.0,-3.0,17.0,...,11.0,17.0,14.0,3.0,14.0,11.0,13.0,12.0,3.0,5.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36571.0,19428.0,845.0,136.0,10099.0,120.0,529.0,2468.0,253.0,228.0
93,60.0,80.0,45.0,70.0,50.0,60.0,60.0,45.0,65.0,65.0,...,50.0,56.0,54.0,44.0,54.0,43.0,45.0,49.0,49.0,59.0
94,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
95,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0


In [121]:
vif_df, selected_features = select_features(features_encoded, 100)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

Test Data Accessing & Cleaning

In [3]:
import requests
#get current season data from FPL API endpoints and identify the keys
fpl_base_url = 'https://fantasy.premierleague.com/api/'
current_season = requests.get(fpl_base_url+'bootstrap-static/').json()
#json = r.json()
current_season.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [4]:
#create dataframes for the current season dictionary keys for data exploration
#- Contains summary of Gameweek data
events_df = pd.DataFrame(current_season['events']) #
phases_df = pd.DataFrame(current_season['phases']) #Shows calendar months for game weeks
teams_df = pd.DataFrame(current_season['teams'])
players_df = pd.DataFrame(current_season['elements'])
element_stats_df = pd.DataFrame(current_season['element_stats'])
element_types_df = pd.DataFrame(current_season['element_types'])

In [6]:
players_df.shape

(624, 67)

#Code to extract weekly game data for all active players in current season
for x in players_df.index :
    print(x)
    player_id = players_df.id[x]
    url = f'https://fantasy.premierleague.com/api/element-summary/{player_id}/'
    r = requests.get(url)
    json = r.json()
    json_history_df = pd.DataFrame(json['history'])
    json_history_past_df = pd.DataFrame(json['history_past'])

       
    if x == 0 :
        all_history_df = json_history_df
        all_history_past_df = json_history_past_df
    else : 
        all_history_df = all_history_df.append(json_history_df)
        all_history_past_df = all_history_past_df.append(json_history_past_df)

In [7]:
all_history_df.shape #Test Data

(3579, 31)

In [8]:
all_history_past_df.shape #Past Seasons Data

(2151, 22)

In [10]:
#Code to save the all players game week data in current and past seasons to csvs
all_history_df.to_csv('/home/laniolao/fpl/FantasyPremierLeague/current_season.csv')
all_history_past_df.to_csv('/home/laniolao/fpl/FantasyPremierLeague/past_seasons.csv')

In [26]:
#Map the team names and the player positions into the players_df dataframe
teams_now=dict(zip(teams_df.id, teams_df.short_name))
positions=dict(zip(element_types_df.id, element_types_df.singular_name_short))
players_df['club_name'] = players_df['team'].map(teams_now)
players_df['position'] = players_df['element_type'].map(positions)

In [27]:
#Update the club names from abbreviations to full names
players_df["club_name"].replace({'ARS': 'Arsenal', 'AVL': 'Aston Villa', 'BOU': 'Bournemouth', 'BRE': 'Brentford', 'BHA': 'Brighton', 'CHE': 'Chelsea', 'CRY': 'Crystal Palace', 'EVE': 'Everton', 'FUL': 'Fulham',
       'LEI': 'Leicester', 'LEE': 'Leeds', 'LIV': 'Liverpool', 'MCI': 'Man City', 'MUN': 'Man Utd', 'NEW': 'Newcastle', 'NFO': 'Nottingham Forest', 'SOU': 'Southampton', 'TOT': 'Spurs',
       'WHU': 'West Ham', 'WOL': 'Wolves'}, inplace=True)
players_df.club_name

0      Arsenal
1      Arsenal
2      Arsenal
3      Arsenal
4      Arsenal
        ...   
619     Wolves
620     Wolves
621     Wolves
622     Wolves
623     Wolves
Name: club_name, Length: 624, dtype: object

In [29]:
players_df['name'] = players_df['first_name'] + ' ' + players_df['second_name']

In [30]:
players_df[['first_name', 'second_name', 'name','club_name', 'minutes', 'form',  'bonus', 'bps', 'total_points', 'value_season', 'value_form' ]].head()

,first_name,second_name,name,club_name,minutes,form,bonus,bps,total_points,value_season,value_form
0,Cédric,Alves Soares,Cédric Alves Soares,Arsenal,0,0.0,0,0,0,0.0,0.0
1,Granit,Xhaka,Granit Xhaka,Arsenal,537,4.8,2,113,26,5.2,1.0
2,Mohamed,Elneny,Mohamed Elneny,Arsenal,90,0.4,0,15,2,0.5,0.1
3,Rob,Holding,Rob Holding,Arsenal,3,0.4,0,8,2,0.5,0.1
4,Thomas,Partey,Thomas Partey,Arsenal,270,1.0,0,39,8,1.7,0.2


In [ ]:
#get current season fixtures from FPL API endpoint and create Dataframe

current_season_fixtures = requests.get(fpl_base_url+'fixtures/').json()
fixtures_df = pd.DataFrame(current_season_fixtures)
fixtures_df.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2292810,1,True,True,1,2022-08-05T19:00:00Z,90,False,True,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,2,74911
1,2292813,1,True,True,4,2022-08-06T11:30:00Z,90,False,True,12,2.0,9,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,74914
2,2292811,1,True,True,2,2022-08-06T14:00:00Z,90,False,True,2,0.0,3,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,2,74912
3,2292814,1,True,True,5,2022-08-06T14:00:00Z,90,False,True,20,1.0,11,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,74915
4,2292816,1,True,True,7,2022-08-06T14:00:00Z,90,False,True,16,0.0,15,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,3,74917


In [31]:
df_allseasons.columns

Index(['Unnamed: 0', 'season_x', 'name', 'position', 'team_x', 'assists',
       'bonus', 'bps', 'clean_sheets', 'creativity', 'element', 'fixture',
       'goals_conceded', 'goals_scored', 'ict_index', 'influence',
       'kickoff_time', 'minutes', 'opponent_team', 'opp_team_name',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'team_a_score', 'team_h_score', 'threat',
       'total_points', 'transfers_balance', 'transfers_in', 'transfers_out',
       'value', 'was_home', 'yellow_cards', 'GW'],
      dtype='object')

In [14]:
all_history_df.columns

Index(['element', 'fixture', 'opponent_team', 'total_points', 'was_home',
       'kickoff_time', 'team_h_score', 'team_a_score', 'round', 'minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'value', 'transfers_balance', 'selected',
       'transfers_in', 'transfers_out'],
      dtype='object')

In [37]:
all_history_df.head(10)

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,bps,influence,creativity,threat,ict_index,value,transfers_balance,selected,transfers_in,transfers_out
0,1,1,7,0,False,2022-08-05T19:00:00Z,0,2,1,0,...,0,0.0,0.0,0.0,0.0,45,0,23970,0,0
1,1,11,10,0,True,2022-08-13T14:00:00Z,4,2,2,0,...,0,0.0,0.0,0.0,0.0,44,-5169,24193,1361,6530
2,1,21,3,0,False,2022-08-20T16:30:00Z,0,3,3,0,...,0,0.0,0.0,0.0,0.0,44,-4337,20960,879,5216
3,1,31,9,0,True,2022-08-27T16:30:00Z,2,1,4,0,...,0,0.0,0.0,0.0,0.0,43,-2988,18825,577,3565
4,1,41,2,0,True,2022-08-31T18:30:00Z,2,1,5,0,...,0,0.0,0.0,0.0,0.0,43,-1611,17790,405,2016
5,1,56,14,0,False,2022-09-04T15:30:00Z,3,1,6,0,...,0,0.0,0.0,0.0,0.0,42,-1207,17133,516,1723
0,3,1,7,2,False,2022-08-05T19:00:00Z,0,2,1,90,...,12,16.6,15.0,2.0,3.4,50,0,48303,0,0
1,3,11,10,12,True,2022-08-13T14:00:00Z,4,2,2,90,...,35,54.8,25.5,28.0,10.8,50,-629,65418,9001,9630
2,3,21,3,6,False,2022-08-20T16:30:00Z,0,3,3,87,...,25,25.8,34.0,6.0,6.6,50,112040,216726,137326,25286
3,3,31,9,2,True,2022-08-27T16:30:00Z,2,1,4,90,...,9,8.2,25.0,12.0,4.5,50,42760,267951,77459,34699


In [38]:
fixtures_df.head(10)

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2292810,1,True,True,1,2022-08-05T19:00:00Z,90,False,True,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",3,2,74911
1,2292813,1,True,True,4,2022-08-06T11:30:00Z,90,False,True,12,2.0,9,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,2,74914
2,2292811,1,True,True,2,2022-08-06T14:00:00Z,90,False,True,2,0.0,3,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,2,74912
3,2292814,1,True,True,5,2022-08-06T14:00:00Z,90,False,True,20,1.0,11,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,74915
4,2292816,1,True,True,7,2022-08-06T14:00:00Z,90,False,True,16,0.0,15,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,3,74917
5,2292817,1,True,True,8,2022-08-06T14:00:00Z,90,False,True,17,1.0,18,4.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,74918
6,2292812,1,True,True,3,2022-08-06T16:30:00Z,90,False,True,6,1.0,8,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,74913
7,2292815,1,True,True,6,2022-08-07T13:00:00Z,90,False,True,4,2.0,10,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,74916
8,2292818,1,True,True,9,2022-08-07T13:00:00Z,90,False,True,5,2.0,14,1.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,4,74919
9,2292819,1,True,True,10,2022-08-07T15:30:00Z,90,False,True,13,2.0,19,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",5,3,74920


In [17]:
all_history_past_df.columns # Does not really provide any new information to us

Index(['season_name', 'element_code', 'start_cost', 'end_cost', 'total_points',
       'minutes', 'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index'],
      dtype='object')

In [23]:
teams_df.columns

Index(['code', 'draw', 'form', 'id', 'loss', 'name', 'played', 'points',
       'position', 'short_name', 'strength', 'team_division', 'unavailable',
       'win', 'strength_overall_home', 'strength_overall_away',
       'strength_attack_home', 'strength_attack_away', 'strength_defence_home',
       'strength_defence_away', 'pulse_id'],
      dtype='object')

In [39]:
fixtures_df.columns

Index(['code', 'event', 'finished', 'finished_provisional', 'id',
       'kickoff_time', 'minutes', 'provisional_start_time', 'started',
       'team_a', 'team_a_score', 'team_h', 'team_h_score', 'stats',
       'team_h_difficulty', 'team_a_difficulty', 'pulse_id'],
      dtype='object')

In [24]:
teams_df.draw.describe()

count    20.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: draw, dtype: float64

In [25]:
teams_df.head()

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1200,1270,1160,1230,1160,1240,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1090,1100,1110,1110,1070,1090,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1050,1050,1070,1110,1040,1070,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,None,False,0,1090,1100,1120,1150,1140,1170,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,None,False,0,1100,1090,1170,1160,1150,1170,131
